In [18]:
from pygments import highlight
from pygments.lexers import CLexer
from pygments.lexers.python import PythonLexer
from pygments.lexers.make import MakefileLexer
from pygments.formatters import HtmlFormatter
import IPython

def get(path):
    if path.endswith(".c"):
        lexer = CLexer()
    elif path.endswith(".py"):
        lexer = PyLexer()
    elif "Makefile" in path:
        lexer = MakefileLexer()
    with open(path) as f:
        code = f.read()
        return '<style type="text/css">{}</style>{}'.format(
            HtmlFormatter().get_style_defs('default'),
            highlight(code, lexer, HtmlFormatter()))
    
# Clean all previous output
!make -f Makefile.x86 clean

rm -f helloworld


# Bare Metal Deployment on RISC-V Host core

The Diana system-on-chip (SoC) is based on [pulpissimo](https://github.com/pulp-platform/pulpissimo) and comes with three main parts:
1. An open-source RISC-V core. In this case, a [**RI5CY**](https://github.com/openhwgroup/cv32e40p) core with the `RV32IMFCXpulp` ISA.
2. A Digital accelerator named **SOMA** with a 16-by-16 PE array supporting e.g. Conv2D operations at 8 bit precision
3. An analog/mixed-signal Compute-in-memory (CIM) core named **ANIA** with a 1152-by-512 PE array supporting e.g. Conv2D operations on 7-bit inputs and ternary weight values (\[-1,0,1\]).

In this tutorial you will learn how to:
* Write a simple "hello world" program on Diana's RISC-V core
* Step through a program with `gdb` on the RISC-V core.
* Cross-compile your C code for deployment on the RISC-V core with `gcc` from the pulp RISC-V GCC toolchain.
* Perform memory managemement for the SoC
* Troubleshoot standard errors and common problems

This tutorial assumes some familiarity with various aspects of C programming:
* C programming
* Basic knowledge of GCC compilation options
* Building C code with GNU `make` and `Makefile`s
* Static and dynamic memory management in C (e.g. `malloc` and `free`)
* Familiarity with a debugger like `gdb`, `lldb`, `pdb` or debuggers found in common IDE's.

## Writing a simple "hello world" program

Writing programs on a platform like Diana is quite different from writing a program on your average desktop platform:
* Diana's code is running *bare-metal*, meaning there is *no OS* on the Diana platform. Your program is directly running on the RISC-V core with no operating system in between.
* You can not attach a screen to Diana, nor can you SSH into Diana. You can only interface with Diana over *UART* or *JTAG* (with the help of `gdb`). 
* Diana has very few memory (*only 512 kB!*) to work with and has no file system.
* Diana has *no hardware caches*, but only *scratchpad memories*, meaning that all data caching has to be controlled by the programmer/compiler.
* Diana's host core works with the quite recent and fully open-source *RISC-V ISA* as opposed to the more traditional x86 or ARM ISA's.

### Writing a simple hello world on X86:

Let's write a basic C program to illustrate how to work with this platform

In [17]:
IPython.display.HTML(get("helloworld.c"))

In [16]:
IPython.display.HTML(get("Makefile.x86"))

In [20]:
!make -f Makefile.x86 all

gcc helloworld.c -o helloworld


Now you can run the `helloworld` binary:

In [21]:
!./helloworld

hello world!


This binary is not deployable on Diana though! Given the earlier comments, do you know what's wrong?

### What is wrong with this example?

A few things are wrong with this binary:
1) It's **not written for a `RISC-V`** machine,`readelf -h helloworld` reveals that it's in fact a **`X86-64`** binary (see the `Machine` entry):

In [22]:
!readelf -h helloworld

ELF Header:
  Magic:   7f 45 4c 46 02 01 01 00 00 00 00 00 00 00 00 00 
  Class:                             ELF64
  Data:                              2's complement, little endian
  Version:                           1 (current)
  OS/ABI:                            UNIX - System V
  ABI Version:                       0
  Type:                              DYN (Shared object file)
  Machine:                           Advanced Micro Devices X86-64
  Version:                           0x1
  Entry point address:               0x1060
  Start of program headers:          64 (bytes into file)
  Start of section headers:          14720 (bytes into file)
  Flags:                             0x0
  Size of this header:               64 (bytes)
  Size of program headers:           56 (bytes)
  Number of program headers:         13
  Size of section headers:           64 (bytes)
  Number of section headers:         31
  Section header string table index: 30


2) Even though Diana **doesn't support an OS**, GCC has wrapped it inside **Linux** startup (`_start`) functions, which can in this case be seen from the entry point address (see previous cell) by *disassembling* the binary with `objdump`:

In [24]:
!objdump -S --start-address=0x1060 --stop-address=0x1090 helloworld


helloworld:     file format elf64-x86-64


Disassembly of section .text:

0000000000001060 <_start>:
    1060:	f3 0f 1e fa          	endbr64 
    1064:	31 ed                	xor    %ebp,%ebp
    1066:	49 89 d1             	mov    %rdx,%r9
    1069:	5e                   	pop    %rsi
    106a:	48 89 e2             	mov    %rsp,%rdx
    106d:	48 83 e4 f0          	and    $0xfffffffffffffff0,%rsp
    1071:	50                   	push   %rax
    1072:	54                   	push   %rsp
    1073:	4c 8d 05 66 01 00 00 	lea    0x166(%rip),%r8        # 11e0 <__libc_csu_fini>
    107a:	48 8d 0d ef 00 00 00 	lea    0xef(%rip),%rcx        # 1170 <__libc_csu_init>
    1081:	48 8d 3d c1 00 00 00 	lea    0xc1(%rip),%rdi        # 1149 <main>
    1088:	ff 15 52 2f 00 00    	callq  *0x2f52(%rip)        # 3fe0 <__libc_start_main@GLIBC_2.2.5>
    108e:	f4                   	hlt    
    108f:	90                   	nop


3. We can just read the output from the comfort of the terminal. As stated earlier, this is not possible with Diana. 
   
   In fact, with the current lab setup, it is not even possible to read from `printf();` statements at all.
  